In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys, io

Using TensorFlow backend.


# Load & Prep data

In [2]:
path = 'data/tale_of_two_cities.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

corpus length: 776702
total chars: 62
nb sequences: 258888


In [3]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Build model

In [4]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(256))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [6]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [7]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Train model

In [8]:
model.fit(X, y, batch_size=128, epochs=1, callbacks=[print_callback])

Epoch 1/1
258816/258888 [============================>.] - ETA: 0s - loss: 3.1981
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "the safest way of life, involving the le"
the safest way of life, involving the lenni nnn nnnini innnnnnn nnn nnn i nnnnnnninnnnnnnin nn  nnnnnnnnnininn nnnninnnnnnnn nnnnnnnnnnninnnnnnnnnninnnn  innnn nn nnnnnnnnnnnn nn ninnn  ninnnnninnnn nninn ninnnnnninnnnnnnnni nninn nn nnnnnnnnnninnnninnnninninin nnnnnn nn nnnnnnnnnnnnnnn  n nnninnni nnnnnn innnnnnnnnninnnninn nnnnnnnnin ninnnnnin innnnnnnnnni n nnn i,innninn innnnnnnnnni nnnnnninninnnii nn ninnnnni nnnnnnnnnnnnnnnnnnnnn 
----- diversity: 0.5
----- Generating with seed: "the safest way of life, involving the le"
the safest way of life, involving the le
n,niinnnnn onnnnnii,ihninnynlynieni,ni n ,in i nntninunii h,,i,n ncinc i nnndonini cnnnnnnen
tnn iinnyrns yminni nnnynn nsn in ih ,,innnn nnin ia nnn,nn nnnnii nnn nni i,hawni,,i.nnnninnnoh   i  i s n ihh o,ahn ei n

In [9]:
model.fit(X, y, batch_size=128, epochs=4, callbacks=[print_callback])

Epoch 1/4
258816/258888 [============================>.] - ETA: 0s - loss: 3.1979
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "d took up her knitting. the
visitor watc"
d took up her knitting. the
visitor watc     

/Users/jeremyvanvalkenburg/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


               o o    r                                               o       o         o         w               o                              o                    r          w r                 h             r      o               oe                    r     o               ,        h                                                                                o                          
----- diversity: 0.5
----- Generating with seed: "d took up her knitting. the
visitor watc"
d took up her knitting. the
visitor watc  o,o , wo hwd  h   rw  rdr  h  r  dro  r   hr r       ohow o  ny         rs  r r  g ,w o   o,  o     , w onw   
   wv rwwwhe    o oa hi r    r w r oo  ohr ora,ws h  o  o v o rr  w     roso  rwrr rwodnoa oa s  no   oe ,,od t iwnhre  n whnee     er l w o          o    ow  ww t  o,, us  o w toed rw or or wei rw rwr deoo  y o ow a  e   ,   o w  he e r  w,   o w s wh oo    w r ,o   mtnhrr   r    hrw o
----- diversity: 1.0
----- Generating with seed: "d took up her knittin

 tosibe ,iit tco.tuytti tetetctctfbson dftfctuvstwenlt frbtped
icdtb t f-tfrn
ddtttochsm;ttb i t bt t uttcitchhdarttntcns-bllw
tg thbttes”ntvtttcfta ttlts f dtit
.sceftdtb
tsthsttitsttctavttrrctutd tlutstoirsts fte?sl-d
yc
rattttldtnnd. naocwdclttttoctuttbuigf
c-ttltgidc,   tttttdtctt tettuudsc
lotst
----- diversity: 1.2
----- Generating with seed: "ve it characterised for me by any
gentle"
ve it characterised for me by any
gentletnttucouatst ctd  dtdnw
cs- tb  asttbt
b rohngt-t g-t t y
lf e  fstutucciec tels,.t.metthttcditlhgbbvt

ugir-cc
wohlwtauttsiacdgl f.rtrtnwtm titdbfssncct tcuottldtcsevweldftd tfmr.utr-.a-nstlu ttddtte dtotdaitlgosdneitcdcttd
tc
 etl-ttefvtfndct


thlrncfraiu-tuttn totne n
buftttat fsdnc
cvtco m oafpui
bttdtscictfc
,rtlb.titbot!
kdtb-ao  .ttl b dt o.cl .tco
r cytbgcntpgnttr ottcetfosrbtbpertscdtidd
258888/258888 [==============================] - 2541s 10ms/step - loss: 3.1989
